# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [9]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import sys

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/rambino/dev/DataEngineering_Udacity/Projects/DataModelingWithCassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

#### Notes:

The code block below was provided by Udacity.
It looks like the CSV files are being read-in and then re-parsed into a new file which (a) clears out missing data and (b) only loads in necessary columns.

**Note:** If I were to re-write this, I *really* don't like the use of numerical indexes to write the new data file. I don't see why we don't take advantage of Pandas to read the data into a DataFrame, and then use `itertuples` to iterate over it and write the new file.

**To Do:** Come back to this script after completing rest of code & try to transform this code

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [53]:
# check the number of rows in your csv file
#df = pd.read_csv('event_datafile_new.csv')

#df.groupby('sessionId').agg({'userId':['nunique']})
df.groupby('sessionId')['userId'].nunique().describe()

# with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
#     print(sum(1 for line in f))

count    776.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: userId, dtype: float64

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpeg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [119]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
contacts = ['127.0.0.1']
port = 9042

from cassandra.cluster import Cluster
cluster = Cluster(contact_points = contacts, port = port)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [120]:
# TO-DO: Create a Keyspace
try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS music_sparkify_cassandra
    WITH REPLICATION =
    {'class':'SimpleStrategy','replication_factor':'1'}
    ''')
    
except Exception as e:
    print(e)


#### Set Keyspace

In [121]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('music_sparkify_cassandra')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
- So filtering on `sessionId` and `itemsInSession`
- Note: Using `df.groupby('sessionId')['userId'].nunique().describe()`  with pandas it seems that each sessionId is unique - even across users - so there is no need to use both `userId` and `sessionId` as composite primary keys- because `sessionId` already can only be 1 user
- The table below uses `sessionId` as the partitioning key so that each partition represents a unique session. Within those sessions, items are ordered by `itemsInSession` which shows the order in which songs were played (I think).

```
CREATE TABLE IF NOT EXISTS songs_by_session
(sessionId int,
itemInSession int,
artist text,
song text,
length float,
PRIMARY KEY (sessionId, itemsInSession))

```

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- So filtering on both `userId` AND `sessionId`. This to me doesn't make any sense. As I wrote above, `sessionId` is already uniquely-identifying users, so there is no reason to also filter by `userId` that I can see.
- The table below uses `userId` and `sessionId` as a composite partitioning key so that each partition again represents a unique session, each of which is again sorted by the clustering key `itemInSession` (since we need to sort based on this value).
- Since `userId` is probably not needed, it would be a good idea to probably combine this table with the one above, but since the query above might *not* specify `userId`, we can't necessarily include it as a partitioning key for the purpose of this project

```
CREATE TABLE IF NOT EXISTS songsUser_by_sessionUser
(sessionId int,
userId int,
itemInSession int,
artist text,
song text,
firstName text,
lastName text
PRIMARY KEY ((sessionId, userId), itemInSession))
```

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- So we need data organized by individual songs. It might be kind of inefficient to partition by each song, since that would mean millions of partitions at scale, and each partition could grow in an unbounded way (no limit to # of song plays).
- The basics of what I need are `firstName`, `lastName` and probably `userId` (since names could repeat). Then of course `song`. The partition key would then be `song` and the clustering key `userId` (or jointly `firstName` and `lastName`). This model would NOT record unique listens (which would be overwritten), but that seems to be o.k. based on the query.
- **NOTE:** The analyst using this query probably is talking about a specific song (from a particular artist) - partitioning by only `song` would group song data *as long as the song name is the same* - which is quite misleading. Therefore, we probably need to **ENFORCE** using artist in this query - so I'm including it as a partition key

```
CREATE TABLE IF NOT EXISTS users_by_song
(userId int,
firstName text,
lastName text,
song text,
artist text,
PRIMARY KEY ((song, artist), userId))
```



In [122]:
'''
Note: Re-designed this code so that it's more reusable for other insert queries / tables.
Includes re-typing values in data stream according to attributes_dict values
'''

def data_insert(table,attributes_dict):
    '''reads in file below and inserts into user-given Cassandra table with user-specified attributes'''
    file = 'event_datafile_new.csv'

    query = make_insert_query_string(table,attributes_dict)

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        columns = next(csvreader) # record & skip col names
        for line in csvreader:
            values = get_insert_values(columns, attributes_dict,line)
            session.execute(query, tuple(values))

def make_insert_query_string(table, attributes):
    '''Creates basic insert query string for Cassandra given table name and list of values'''
    attributes_str = ", ".join(attributes)
    values_str = ", ".join(["%s"]*len(attributes))
    query = f"INSERT INTO {table} ({attributes_str}) VALUES ({values_str})"
    return query

def get_insert_values(columns, attribute_dict, data):
    '''
    Casts data values as type defined by 'attribute_dict'.
    Uses 'columns' to ensure that the correct index of 'data' is being used.
    Requires that all values in attribute_dict.keys() match an element in 'columns'
    '''
    return [func(data[columns.index(attr)]) for attr,func in attribute_dict.items()]


### Query #1

In [123]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

session.execute(
    '''
    CREATE TABLE IF NOT EXISTS songs_by_session
    (sessionId int,
    itemInSession int,
    artist text,
    song text,
    length float,
    PRIMARY KEY (sessionId, itemInSession))
    '''
)

#session.execute("DROP TABLE songs_by_session")

In [124]:
#Query 1: Inserting Data
attributes_dict = {
    'sessionId':int,
    'itemInSession':int,
    'artist':str,
    'song':str,
    'length':float
}
table = 'songs_by_session'

data_insert(table,attributes_dict)

#### Do a SELECT to verify that the data have been inserted into each table

In [125]:
#Query 1: Testing Insert
result = session.execute(
    '''SELECT * FROM songs_by_session WHERE sessionId = 193'''
)
for row in result:
    print(row)

Row(sessionid=193, iteminsession=1, artist='Greg Laswell', length=251.08853149414062, song='Comes And Goes ( In Waves )')
Row(sessionid=193, iteminsession=3, artist='Pavement', length=99.16036224365234, song='Mercy:The Laundromat')
Row(sessionid=193, iteminsession=4, artist='Sixx A.M.', length=280.5807800292969, song='Courtesy Call (Explicit)')
Row(sessionid=193, iteminsession=5, artist='Leonard Cohen', length=267.67626953125, song="I'm Your Man")


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Query #2

In [126]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
session.execute(
    '''
    CREATE TABLE IF NOT EXISTS songsUser_by_sessionUser
    (sessionId int,
    userId int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY ((sessionId, userId), itemInSession))
    '''
)         

#session.execute("DROP TABLE songsUser_by_sessionUser")

In [127]:
#Query 2:
attributes_dict = {
    'sessionId':int,
    'userId':int,
    'itemInSession':int,
    'artist':str,
    'song':str,
    'firstName':str,
    'lastName':str
}
table = 'songsUser_by_sessionUser'

data_insert(table,attributes_dict)

In [128]:
#Query 2: Testing Insert
result = session.execute(
    '''SELECT * FROM songsUser_by_sessionUser WHERE userId = 10 AND sessionId = 182'''
)
for row in result:
    print(row)

Row(sessionid=182, userid=10, iteminsession=0, artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On")
Row(sessionid=182, userid=10, iteminsession=1, artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000')
Row(sessionid=182, userid=10, iteminsession=2, artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer')
Row(sessionid=182, userid=10, iteminsession=3, artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


### Query 3

In [129]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute(
    '''
    CREATE TABLE IF NOT EXISTS users_by_song
    (userId int,
    firstName text,
    lastName text,
    song text,
    artist text,
    PRIMARY KEY ((song, artist), userId))
    '''
)         

#session.execute("DROP TABLE users_by_song")

                    

In [130]:
#Query 3:
attributes_dict = {
    'userId':int,
    'firstName':str,
    'lastName':str,
    'artist':str,
    'song':str
}

table = 'users_by_song'

data_insert(table,attributes_dict)

In [131]:
#Query 3:
result = session.execute(
    "SELECT firstName,lastName FROM users_by_song WHERE song = 'All Hands Against His Own' AND artist = 'The Black Keys'"
)

for row in result:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [132]:
## TO-DO: Drop the table before closing out the sessions

session.execute("DROP TABLE songs_by_session")
session.execute("DROP TABLE songsUser_by_sessionUser")
session.execute("DROP TABLE users_by_song")

### Close the session and cluster connection¶

In [133]:
session.shutdown()
cluster.shutdown()

In [ ]:
#Changes needed:
'''
1. Implement my change to data file manipulation, so that we work with tuples + pandas
2. Change parsing of file to be unicode rather than UTF-8 (some foreign characters mis-interpreted)
3. Re-write the rationale of my solutions and put in markdown file
'''